<a href="https://colab.research.google.com/github/tanaydangaich/Deep-Learning/blob/main/Myanmar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install wandb
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#os.environ['KMP_DUPLICATE_LIB_OK']='True'


import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import os
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import csv

import pandas as pd
import numpy as np
import wandb
import argparse
from scipy.stats import linregress
import random
from sklearn.preprocessing import StandardScaler
#from utils import convert_train_test_split, split_train_val_cv
#from Myanmar_dataset_split import myanmar_test_ix, myanmar_train_ix

def split_train_val_cv(train_ix, cv):
    result = []
    for i in range(cv-1):
        result.append(train_ix[int(len(train_ix)/cv*i):int(len(train_ix)/cv*(i+1))])
    result.append(train_ix[int(len(train_ix)/cv*(i+1)):])
    return result

myanmar_train_ix=[1433, 10933, 8652, 6759, 188, 2708, 7480, 9110, 3943, 1412, 7332, 6094, 8998, 1157, 6488, 6880, 7929, 9151, 8529, 3606,
          8263, 10112, 10040, 7310, 7909, 10883, 8448, 3822, 6906, 8024, 1753, 4264, 7177, 7895, 149, 4023, 2881, 8647, 2755,
          11008, 11058, 7424, 8530, 347, 3788, 8219, 10101, 3937, 8991, 2117, 7392, 8175, 1506, 7217, 2207, 7305, 6738, 9265,
          6885, 7983, 1554, 7593, 2363, 7396, 4027, 7926, 10084, 11954, 11870, 7192, 6275, 6174, 8212, 6839, 11798, 6093,
          8055, 7211, 1411, 7601, 3917, 6827, 6374, 8959, 8572, 1600, 10561, 8554, 7052, 7464, 5217, 8685, 6897, 225, 8782, 4817,
          3867, 6032, 9063, 1935, 1933, 8066, 6558, 7061, 7795, 8730, 6761, 6817, 6423, 1418, 11047, 3908, 4835, 2395, 5132, 1355,
          4010, 6653, 9971, 8049, 7941, 6752, 6340, 6891, 8536, 7646, 7719, 4762, 7375, 7459, 11078, 8164, 6788, 7738, 11591,
          2470, 7655, 1751, 2136, 6853, 8188, 3614, 6520, 6673, 5714, 1886, 1324, 3694, 8749, 2394, 5349, 7294, 9033, 7833, 3793,
          1327, 7390, 6841, 6921, 11013, 8266, 3960, 3989, 4642, 2040, 6316, 6466, 11836, 6096, 6385, 1311, 1643, 8360, 7436,
          7246, 7611, 1694, 6175, 2236, 9123, 10129, 8472, 4711, 2385, 303, 8825, 8736, 9476, 7468, 2678, 7435, 2745, 3573, 2617,
          7308, 1628, 9985, 6780, 6629, 3837, 3730, 2349, 4349, 6386, 7770, 8148, 3949, 7411, 6784, 2579, 6237, 1462, 8048, 8497,
          8149, 2199, 7555, 7925, 2752, 10903, 7402, 8857, 10157, 8144, 6878, 6602, 7053, 4670, 7115, 11860, 10889, 8671, 7311,
          2373, 7588, 10550, 8907, 9294, 38, 2143, 7619, 1617, 9270, 2823, 7753, 8829, 7807, 7768, 10538, 2234, 1579, 11034, 7553,
          8096, 3713, 10491, 7670, 9097, 1268, 2010, 6869, 6249, 1490, 11851, 8101, 8183, 6373, 6772, 2622, 7706, 9149, 6353, 4860,
          4043, 3854, 1307, 1357, 3666, 8061, 6317, 8956, 2212, 1764, 8674, 8623, 9113, 3723, 8138, 7066, 8221, 6819, 4254, 10952,
          9085, 11140, 8161, 8706, 8211, 2088, 9049, 3629, 8067, 2166, 1689, 6478, 2289, 1274, 4292, 4357, 3745, 6764, 1465, 1586,
          2699, 2720, 8847, 8631, 8794, 9170, 6521, 2512, 6345, 8591, 3115, 265, 6797, 9968, 4654, 8723, 3786, 7600, 11083, 10978,
          4230, 8374, 9119, 8740, 5123, 222, 6100, 8679, 8252, 7968, 7154, 7269, 10517, 2450, 7470, 4263, 2449, 1308, 2169, 10964,
          7347, 10010, 9099, 8531, 6756, 7256, 7286, 3955, 7721, 1374, 7077, 2327, 1295, 6523, 9083, 2386, 2637, 11733, 3048, 1749,
          2217, 3570, 10053, 7687, 1394, 10991, 2085, 6496, 6762, 7339, 7266, 6446, 1197, 3654, 2794, 8043, 6736, 3739, 4243, 7783,
          381, 3761, 7431, 2316, 424, 10002, 3835, 2122, 1320, 8143, 4810, 10899, 3973, 2080, 1921, 6399, 2684, 4673, 9041, 3759,
          1468, 6804, 2785, 6771, 6465, 8193, 4763, 3670, 3775, 8788, 7701, 2451, 1223, 11032, 8013, 1932, 7641, 1623, 7395, 10977,
          8665, 11124, 7537, 2347, 6178, 8222, 7961, 1509, 1198, 4025, 10035, 118, 8899, 11081, 7307, 3642, 1467, 7723, 2801, 7695,
          4649, 2192, 160, 1401, 1111, 7199, 7051, 1687, 7454, 7582, 2138, 7134, 9188, 3617, 10027, 1994, 7826, 9976, 6219, 10409,
          6164, 6710, 11129, 7858, 8408, 7872, 7191, 2043, 7242, 8523, 8022, 2813, 1583, 2654, 6474, 1688, 10175, 10831, 2331, 4653, 6769, 8090, 5298, 163, 7879, 124, 4041, 7597, 4728, 4275, 4717, 2184, 7258, 8424, 10528, 7740, 4363, 2674, 7839, 7763, 4273, 11071, 7951, 2767, 6613, 6838, 8186, 7561, 2333, 7501, 1435, 8082, 8152, 2339, 8047, 9974, 6158, 3647, 9105, 5128, 6557, 8661, 7, 4337, 10970, 1963, 6083, 2189, 7086, 1627, 6702, 3632, 8743, 8079, 10700, 4765, 3912, 2377, 7179, 8912, 8974, 8819, 6296, 8367, 7386, 5098, 2154, 8431, 11147, 7897, 10507, 10539, 6259, 9000, 5134, 316, 10989, 2821, 7977, 7497, 6176, 2468, 3780, 4815, 4240, 6732, 7471, 7438, 8541, 3784, 8849, 7786, 3770, 7122, 6837, 7484, 4352, 4020, 3947, 2037, 3612, 8250, 7618, 8986, 2427, 8284, 4695, 6325, 7425, 7205, 6918, 1605, 1323, 4336, 3596, 6156, 4644, 7068, 8636, 10945, 3763, 3878, 6504, 2120, 192, 8624, 9979, 7577, 3794, 10877, 7362, 8110, 6106, 2464, 3986, 3743, 3546, 7458, 1496, 7197, 11780, 9353, 8675, 8228, 8549, 6293, 5279, 7287, 8911, 1098, 2194, 4660, 1635, 1592, 3649, 2328, 3636, 4241, 6920, 7383, 6471, 1603, 1530, 2071, 8085, 3843, 2104, 7514, 7499, 1581, 7325, 10548, 4865, 1392, 1247, 1436, 1920, 8635, 405, 3432,
          6793,
          8338,
          3602,
          8413,
          6314, 7155, 7189, 4907, 3705, 8688, 3731, 1446, 2108, 7529, 10535, 1567, 8962, 1283, 8467, 6861, 6375, 2420, 7542, 8581, 1178, 11755, 3827, 7178, 7551, 9038, 8092, 8620, 2277, 9167, 4734, 6086, 6828, 2317, 2431, 7728, 3791, 7586, 8783, 7985, 8150, 6813, 6501, 8477, 4760, 7343, 1221, 11036, 11073, 10485, 6389, 11793, 5694, 10556, 6455, 6217, 10011, 8241, 7469, 3789, 4369, 3911, 2374, 7337, 2822, 7761, 3648, 6860, 6910, 10488, 1693, 1410, 7408, 6481, 1423, 1428, 8045, 6538, 7603, 1597, 11822, 444, 6648, 11811, 2162, 7640, 8640, 7928, 4192, 11045, 4911, 4735, 9318, 1962, 8874, 247, 4035, 10521, 6420, 11813, 7407, 8420, 2313, 9066, 8314, 2393, 8778, 6380, 6892, 8916, 3857, 7288, 1138, 4915, 5129, 2376, 4778, 6795, 7505, 6482, 8025, 7419, 8995, 3532, 1447, 8385, 3803, 9997, 2308, 9023, 1337, 8840, 11819, 3815, 7277, 3939, 8478, 8946, 1225, 2165, 7394, 7747, 2002, 3846, 2268, 1633, 1135, 4696, 4009, 6439, 10828, 5348, 6633, 1938, 8603, 7920, 8983, 204, 10846, 8400, 8185, 10829, 3652, 25, 7472, 10128, 8356, 11039, 2366, 1216, 1500, 7440, 6155, 7571, 4261, 10817, 3962, 1512, 3588, 2714, 6901, 7836, 3655, 6051, 206, 4879, 9285, 6509, 6361, 7859, 1314, 6803, 4058, 2334, 3527, 2766, 8381, 4726, 11827, 9039, 6783, 8969, 2038, 6288, 8700, 7247, 3824, 7766, 11131, 9117, 1312, 9132, 4200, 9141, 7437, 2235, 3669, 9172, 4700, 7147, 8278, 1232, 8627, 11120, 10495, 11100, 8311, 6066, 3659, 1419, 9109, 6054, 8684, 6462, 3877, 8555, 2454, 327, 6313, 2681, 3550, 11015, 7412, 5100, 425, 9137, 10931, 11876, 7982, 1616, 1908, 6812, 2291, 3605, 7340, 9198, 3526, 3688, 6698, 6750, 7450, 5079, 3456, 9069, 4255, 6737, 2805, 1756, 8876, 4788, 102, 1538, 281, 3842, 3700, 1541, 6408, 8724, 8801, 3737, 2465, 8763, 8473, 7950, 1508, 8208, 10551, 2486, 8153, 352, 6205, 6368, 7352, 1883, 8456, 6562, 7604, 3806, 7322, 10920, 7432, 7562, 8607, 7927, 3641, 2360, 7479, 2281, 7112, 7397, 7439, 7755, 11764, 7940, 9981, 11069, 5074, 8060, 10014, 8287, 1159, 8855, 1479, 11040, 10510, 3591, 1478, 7634, 9991, 7996, 6338, 2826, 2721, 7371, 7769, 6023, 11041, 2778, 10557, 344, 10500, 6735, 8687, 6763, 11022, 4356, 2644, 3581, 2305, 10523, 6456, 3704, 2224, 8670, 9161, 7342, 3067, 1251, 8177, 2466, 9972, 3493, 7715, 184, 6526, 1256, 10067, 8020, 7816, 6445, 9078, 1408, 3524, 409, 11142, 9008, 1310, 8267, 8562, 7127, 8544, 4731, 4874, 10044, 8611, 3087, 1169, 7907, 3856, 9990, 7106, 6483, 8083, 8449, 6120, 1144, 11079, 11080, 4138, 8961, 1376, 1610, 8086, 7913, 9268, 8871, 4267, 3626, 3556, 8865, 4201, 7165, 8180, 4656, 11143, 9963, 7589, 8159, 2114, 7648, 6859, 11778, 8957, 11068, 6059, 6779, 6191, 4844, 8103, 1238, 3747, 10993, 1748, 3760, 10546, 1882, 7629, 380, 2276, 9139, 1227, 8949, 1235, 5693, 2686, 10037, 4898, 1217, 7614, 7379, 1729, 4137, 6238, 6630, 20, 8281, 7796, 6079, 7200, 8602, 8253, 4021, 8114, 6843, 7957, 7724, 7526, 7857, 8364, 1894, 3850, 10522, 1218, 7953, 1826, 10867, 4198, 9026, 7517, 8709, 2364, 6132, 6070, 7666, 9156, 3459, 1270, 4091, 3430, 6350, 2700, 3738, 11052, 8051, 4057, 5299, 8235, 2256, 8383, 1584, 1880, 8279, 5063, 10972, 1115, 11834, 8322, 7100, 6714, 8075, 7627, 1730, 8248, 8037, 7159, 3582, 2134, 8984, 2662, 1608, 6336, 8217, 4006, 9057, 11109, 6201, 2487, 1548, 3896, 2152, 4845, 3650, 3560, 6822, 10480, 7781, 2401, 9093, 8077, 8556, 10552, 6908, 8586, 7249, 8833, 6547, 2190, 2368, 8039, 8084, 8732, 6486, 6791, 3571, 6884, 8927, 3925, 7652, 8978, 7581, 8042, 6498, 7157, 8187, 1536, 4714, 8770, 2671, 2089, 2444, 2209, 10115, 8125, 8026, 1249, 1827, 1172, 7531, 6746, 11804, 8775, 11808, 1346, 8686, 7409, 6907, 9047, 10008, 6490, 6330, 2378, 8107, 1964, 3841, 1906, 3611, 8301, 9062, 3590, 6832, 7084, 3661, 3801, 1755, 8260, 2220, 10134, 7074, 8146, 4049, 11035, 7546, 7882, 2178, 8583, 3956, 8932, 6334, 6071, 11043, 6825, 7758, 10036, 10502, 6077, 7887, 6566, 1101, 1892, 1128, 10816, 6742, 8422, 6848, 7782, 6516, 1651, 10408, 8981, 2715, 7881, 6524, 322, 1415, 7578, 10176, 10160, 1588, 2348, 6436, 2438, 2255, 1604, 4679, 8558, 6626, 4887, 6815, 3997, 6184, 6753, 7534, 8796, 2161, 6730, 7117, 7254, 6319, 2075, 8325, 6864, 8720, 10865, 3639, 8610, 2459, 8792, 8476, 1184, 6179, 11006, 2448, 10497, 2652, 353, 3580, 7904, 9130, 8747, 7793, 6917, 2761, 6128, 10070, 7445, 6922, 2218, 7045, 7784, 8797, 3076, 6836, 6396, 3559, 1434, 1566, 11812, 9095, 162, 7323, 8038, 8128, 1414, 7481, 7963, 9351, 10506, 7406, 7993, 8923, 8657, 2323, 8977, 2095, 7196, 7107, 1510, 7387, 1368, 6116, 388, 8016, 2698, 1564, 7693, 3684, 7398, 10844, 1292, 8924, 6870, 8996, 8835, 1359, 7222, 7736, 2066, 8195, 9048, 10169, 1365, 7625, 1613, 8210, 7509, 8136, 9118, 11809, 6866, 7373, 6299, 8522, 10126, 7190, 3971, 7055, 1653, 4207, 11146, 11067, 7965, 8933, 7623, 2796, 10049, 8359, 1212, 311, 4847, 7726, 2275, 3558, 363, 435, 6291, 8557, 3476, 3525, 3088, 3902, 11118, 7813, 10892, 7767, 3593, 6438, 4219, 10508, 8746, 11820, 6760, 2361, 3744, 7120, 8853, 2973, 7151, 1481, 10559, 1493, 6063, 4188, 6395, 8486, 10841, 2641, 7774, 10050, 10915, 6886, 7862, 1338, 7922, 8126, 369, 10141, 8206, 2177, 1503, 3893, 6550, 2818, 4187, 9121, 7615, 6821, 8731, 4997, 1429, 4856, 1765, 7530, 7939, 7657, 2006, 6929, 1576, 10519, 11053, 7725, 4792, 261, 10914, 1208, 3750, 8772, 7984, 5339, 8919, 366, 8246, 8664, 10979, 6257, 7587, 7490, 1577, 183, 3564, 6197, 1489, 1626, 3454, 10997, 3635, 7410, 8465, 2290, 1782, 7336, 6467, 7368, 8779, 1265, 314, 1961, 7811, 8194, 11903, 8312, 8980, 11046, 2284, 7684, 9201, 8011, 10082, 8877, 8454, 4242, 6469, 1181, 9131, 262, 2115, 2707, 1381, 8551, 4268, 8460, 4777, 161, 6749, 2751, 8869, 4072, 7260, 1531, 7976, 4212, 9266, 10038, 8094, 1248, 7302, 8948, 2415, 2326, 1377, 11105, 92, 159, 8097, 2201, 10537, 11776, 3627, 1137, 1333, 3799, 5070, 1491, 7591, 7663, 5347, 7167, 8041, 7585, 10158, 1492, 3749, 4820, 7869, 1438, 7312, 6898, 2139, 7467, 11840, 9988, 244, 10001, 9164, 7669, 4222, 10882, 3448, 9133, 6728, 9293, 7991, 1520, 7674, 3658, 4253, 5069, 8967, 8976, 8390, 8908, 6739, 8323, 6367, 4120, 8831, 7441, 10081, 8982, 3674, 1657, 7949, 3895, 3065, 6655, 260, 6381, 6807, 6344, 8405, 10954, 7521, 6508, 6356, 7313, 10171, 1236, 383, 2954, 4033, 1155, 8897, 7617, 10483, 9089, 1898, 6357, 1495, 9147, 1460, 10890, 3796, 4024, 2694, 1109, 7156, 2613, 8227, 9096, 1645, 7216, 1647, 10926, 10558, 7979, 3553, 7739, 8507, 8233, 11767, 7877, 8588, 6487, 7162, 2440, 7527, 7820, 7844, 7507, 1153, 3934, 2208, 3938, 9184, 8539, 8416, 3707, 9181, 4007, 11075, 1601, 1325, 3663, 9007, 7573, 2476, 6776, 9094, 6347, 7986, 8905, 3968, 10463, 8935, 8239, 8280, 6852, 5216, 6913, 6492, 6518, 9144, 1279, 3987, 3563, 1499, 53, 7675, 6417, 1116, 1958, 6541, 8571, 8904, 8425, 8559, 5954, 6321, 2446, 6544, 4880, 7135, 4677, 8817, 8410, 135, 8010, 1158, 6782, 7366, 7160, 5179, 8729, 10921, 6427, 8255, 7319, 4657, 6801, 10407, 2125, 1226, 1474, 7827, 8343, 8875, 4082, 3682, 1629, 7825, 3462, 127, 8336, 8527, 4221, 7902, 4238, 7855, 7563, 8297, 6757, 6271, 3577, 406, 8672, 6413, 10104, 8612, 8119, 8800, 9073, 3690, 8417, 6458, 8134, 358, 6879, 6740, 7580, 7176, 7274, 7504, 2142, 6731, 8691, 8545, 3587, 8638, 2786, 7898, 6341, 4346, 7883, 7283, 6862, 8274, 7671, 8773, 7170, 6379, 3600, 8692, 7560, 8428, 9166, 2211, 6545, 8822, 9967, 4811, 10958, 8351, 11845, 6305, 11892, 4001, 1649, 1136, 3595, 10124, 6911, 7659, 7933, 11810, 6038, 8622, 2776, 4189, 5101, 8653, 1168, 1551, 9122, 7852, 10504, 1318, 1364, 4085, 7814, 2020, 8353, 7511, 6676, 10545, 7400, 432, 7239, 6800, 1909, 8053, 7164, 9326, 1611, 8044, 443, 11111, 1780, 3572, 10499, 2705, 6026, 9148, 8243, 2787, 2670, 128, 7967, 2975, 6748, 2106, 6594, 9051, 210, 4905, 8165, 1648, 7064, 10843, 4211, 6542, 1105, 3565, 2202, 5970, 1386, 1502, 4794, 67, 2408, 6029, 8880, 4067, 1360, 7891, 7235, 7637, 8613, 11089, 4833, 7376, 7374, 4210, 4076, 8712, 2124, 310, 8762, 1494, 6674, 6067, 7085, 7823, 11901, 9124, 8203, 6090, 2643, 10150, 2406, 3109, 6631, 6904, 7691, 8054, 8178, 8093, 9146, 2098, 8606, 8033, 6522, 6027, 11059, 6258, 8376, 6221, 6874, 7524, 7087, 3916, 9061, 3657, 8021, 7123, 1691, 3630, 8071, 7073, 11018, 7296, 2398, 6808, 9292, 6605, 7136, 4345, 6382, 9028, 3990, 4213, 6308, 7420, 8879, 7116, 4904, 2285, 8040, 7145, 8629, 7540, 6366, 1881, 1537, 9157, 7447, 6323, 8137, 8615, 1766, 8698, 7129, 1660, 8192, 4710, 11065, 10099, 2675, 2167, 3673, 8599, 11774, 7579, 10947, 10102, 94, 10981, 9052, 7414, 2346, 8401, 10910, 9004, 7975, 9006, 7660, 2355, 10110, 2312, 7788, 1809, 2682, 8099, 7538, 8012, 1260, 8419, 10984, 8745, 1241, 8920, 1810, 2469, 9025, 4214, 8968, 7207, 8145, 1690, 1907, 4862, 8299, 7125, 11828, 4766, 8313, 7168, 2471, 9289, 8295, 1399, 7404, 27, 8690, 8098, 11749, 3477, 10071, 2734, 2716, 7427, 6225, 7243, 1396, 8102, 2304, 7354, 8028, 8637, 7193, 9013, 8160, 221, 6849, 6354, 7478, 7702, 6251, 3598, 302, 8268, 2325, 8648, 2137, 9253, 8501, 6820, 11763, 11849, 8873, 136, 1185, 11803, 3656, 1666, 3646, 2254, 8213, 4013, 8095, 7651, 6372, 6226, 2783, 11092, 7812, 8851, 7429, 6875, 2070, 8787, 7487, 7954, 8718, 10830, 7686, 8646, 7631, 10863, 6912, 3695, 6165, 6204, 1485, 2642, 6863, 8754, 3928, 2341, 7749, 3858, 9154, 8373, 7704, 7138, 7210, 7535, 2342, 7341, 8997, 7934, 11097, 10864, 7048, 10512, 8582, 1614, 3667, 6786, 1638, 4232, 2324, 1573, 8332, 10520, 10534, 6303, 6428, 10996, 10182, 8215, 11093, 7936, 7964, 3965, 1557, 5, 7353, 7214, 4016, 6560, 7626, 8068, 1141, 1300, 10103, 7263, 7321, 8326, 239, 11099, 2153, 8050, 1544, 1747, 2825, 7327, 1375, 329, 2164, 2974, 10839, 9067, 11750, 3861, 10064, 11953, 8881, 8264, 7952, 6430, 6215, 7846, 7917, 2191, 8552, 7485, 8027, 8567, 9163, 2118, 7500, 8766, 8104, 1757, 1103, 11126, 8481, 3907, 2306, 7413, 8100, 8184, 5321, 2704, 1122, 8931, 7864, 3555, 7847, 11859, 6703, 1543, 7364, 1257, 7276, 69, 9284, 2226, 1594, 197, 1426, 6858, 3042, 1302, 6414, 2087, 7314, 7762, 9999, 11800, 7741, 10505, 8257, 8965, 2702, 6777, 6604, 1598, 7072, 3509, 8790, 6415, 2663, 11023, 8667, 10490, 8761, 5125, 8791, 3840, 7062, 3623, 2976, 6085, 7780, 8862, 7198, 8858, 7449, 8204, 3681, 7658, 6111, 11761, 7426, 2128, 3748, 8317, 2296, 9150, 3868, 10095, 7046, 3691, 3557, 4325, 3619, 6555, 7644, 1778, 4204, 7525, 7630, 7183, 6039, 7639, 1497, 8169, 7734, 1571, 3820, 8784, 10000, 7654, 8893, 7731, 8218, 8296, 7735, 1556, 4669, 2336, 6616, 1397, 3583, 6261, 8072, 8259, 4353, 1286, 10544, 5131, 1119, 3702, 1398, 3622, 3884, 7255, 8392, 7049, 7908, 7668, 3678, 1264, 7056, 3621, 11865, 6167, 1637, 8711, 217, 7169, 3926, 4256, 1879, 2437, 7365, 6159, 3995, 9288, 7320, 1417, 7873, 8580, 8992, 8282, 6805, 8158, 7778, 6421, 1363, 1959, 8513, 1999, 7594, 3883, 7175, 3924, 3592, 6755, 1384, 8659, 7213, 1523, 6461, 11061, 3097, 305, 10554, 1966, 1639, 1507, 10075, 8493, 6047, 8320, 10976, 6743, 4235, 41, 1621, 9354, 6704, 349, 4834, 3892, 8017, 8289, 8303, 11956, 10145, 6856, 9155, 8242, 5114, 6065, 6448, 8993, 2180, 4077, 7838, 7544, 401, 2370, 7109, 1321, 7930, 47, 4259, 6409, 7990, 4126, 3561, 8370, 10540, 10543, 3676, 6530, 11955, 1281, 2072, 4048, 1133, 10986, 6754, 6846, 95, 1416, 7285, 2238, 3613, 7932, 8697, 2271, 6595, 7536, 1250, 8633, 7434, 317, 1373, 7403, 2397, 7297, 7231, 1555, 6889, 2330, 2789, 10973, 3756, 6433, 4738, 3680, 5364, 7355, 8459, 8972, 1575, 7389, 4205, 2724, 208, 3449, 4216, 6383, 7624, 7808, 10893, 3112, 6877, 7733, 6052, 4705, 7466, 10098, 8111, 3958, 1195, 3435, 7973, 4233, 242, 7506, 3755, 9128, 7532, 7378, 9082, 7488, 8964, 2252, 7417, 3890, 7931, 1532, 8182, 5097, 415, 1261, 10956, 2280, 7377, 1347, 3849, 8032, 1233, 7422, 2369, 4680, 7126, 2638, 15, 392, 10405, 7764, 8654, 6778, 8091, 7994, 4034, 9055, 8645, 7819, 4853, 80, 144, 3809, 2182, 7262, 7282, 8722, 10555, 1606, 7903, 8272, 8669, 8170, 7851, 10181, 7564, 233, 2693, 8682, 7401, 7108, 3616, 8818, 7730, 1504, 7943, 6402, 374, 9143, 9102, 7382, 11748, 6802, 6320, 7065, 10940, 1370, 9322, 8702, 2809, 7098, 6888, 7448, 4331, 3584, 8716, 10060, 3569, 10836, 2433, 2647, 8197, 2258, 7802, 10527, 6122, 6364, 2174, 4260, 7523, 2735, 7316, 8261, 1482, 7824, 8618, 7541, 3811, 1781, 4805, 7694, 10139, 8765, 7081, 8573, 11086, 2432, 7856, 8273, 6450, 8520, 1367, 2144, 2390, 6758, 8883, 4338, 6388, 252, 7418, 7690, 8901, 8087, 1380, 7829, 8073, 4816, 7683, 1997, 1746, 10481, 8896, 7385, 1529, 1937, 11759, 3716, 7384, 2455, 7673, 3819, 8604, 7257, 11076, 7442, 3589, 6872, 8418, 10489, 6137, 8046, 7854, 1472, 8540, 8226, 333, 1160, 7672, 8715, 7356, 7391, 229, 420, 7549, 6733, 2158, 7281, 7568, 3046, 9044, 8009, 4186, 8587, 2462, 10918, 4803, 269, 7914, 9114, 1369, 7502, 3554, 6744, 7722, 8917, 11148, 212, 3604, 10859, 8254, 3701, 8828, 8057, 10092, 114, 7059, 10127, 1125, 2815, 8649, 9358, 122, 3644, 452, 2129, 3988, 8484, 6056, 1545, 7900, 11768, 10904, 58, 8275, 10164, 6411, 2736, 6890, 7834, 6894, 5969, 9225, 1602, 8963, 11838, 7267, 7696, 7628, 2802, 6279, 6712, 7054, 6851, 8952, 7301, 1542, 10501, 2484, 7079, 2009, 2728, 9165, 5066, 168, 6470, 10542, 3844, 10532, 4252, 8966, 1471, 11783, 3734, 3615, 2249, 2259, 103, 6741, 3996, 1278, 6615, 6459, 6794, 1332, 8245, 1361, 2649, 7315, 7137, 6895, 449, 10077, 4239, 10498, 10406, 1528, 6370, 7935, 7944, 3578, 4060, 6883, 323, 3732, 2746, 7139, 3983, 1488, 7633, 6097, 4739, 7172, 6121, 7465, 7132, 1731, 7149, 7818, 1353, 8065, 2195, 325, 10033, 7801, 1779, 6781, 8399, 8689, 11136, 6248, 10513, 1207, 6766, 1179, 10492, 6169, 10753, 6124, 11758, 6700, 3458, 8277, 8695, 1263, 2242, 11866, 4832, 4704, 3576, 7118, 6816, 2103, 7850, 3898, 7298, 6355, 4340, 364, 8133, 4801, 6200, 8878, 6444, 8377, 7088, 6457, 6429, 1437, 351, 2343, 1461, 1189, 1589, 7223, 2792, 8001, 8616, 6850, 7226, 10174, 9074, 7828, 9116, 3653, 2126, 2074, 7303, 2414, 5302, 6432, 235, 2788, 7861, 1533, 64, 9138, 10516, 4699, 6475, 398, 10048, 4651, 7865, 7849, 6537, 7150, 6535, 3692, 2090, 7621, 7359, 8176, 6826, 4191, 8318, 6265, 2416, 4790, 2081, 8619, 6239, 7569, 2300, 2301, 6561, 3520, 8398, 4036, 7078, 4886, 2780, 6037, 7680, 7797, 6716, 6905, 8492, 2625, 1483, 1552, 2196, 9010, 7270, 11144, 1444, 4850, 3957, 9966, 6454, 1631, 2371, 7748, 4913, 7645, 7105, 11753, 8512, 9002, 6245, 1108, 4037, 1147, 8651, 6533, 6619, 11, 8269, 1104, 9100, 4088, 10868, 9101, 7351, 3671, 11744, 2113, 4087, 2442, 436, 2187, 7265, 8495, 8171, 7602, 10845, 9086, 7363, 10835, 7576, 6499, 6072, 3722, 172, 8600, 7799, 9022, 6101, 8844, 2163, 10953, 7947, 2121, 9080, 97, 1130, 3818, 1343, 8089, 9169, 2658, 6785, 198, 3468, 10055, 5596, 2417, 1511, 190, 9964, 5113, 1957, 7317, 1644, 1726, 1771, 10138, 1406, 8140, 1808, 272, 3931, 8584, 8201, 1763, 7677, 6606, 3751, 7810, 2299, 8341, 7840, 3959, 6745, 6195, 1695, 7923, 8002, 11098, 6810, 8117, 11815, 3107, 10030, 7099, 7842, 2082, 6318, 10034, 3457, 7495, 7737, 4746, 6559, 8271, 6435, 3829, 4775, 7665, 7518, 2353, 8947, 11874, 9162, 3455, 7280, 7732, 8131, 11064, 11117, 4217, 8291, 7837, 3608, 8928, 9135, 7510, 5061, 7817, 6767, 6182, 11900, 2804, 7161, 4658, 2412, 2233, 4079, 2387, 7094, 6194, 6424, 4706, 8294, 9112, 10, 336, 6896, 2743, 10549, 8535, 2618, 4000, 7237, 1344, 2723, 2329, 2765, 8838, 7318, 7860, 227, 7133, 2244, 7261, 7884, 6346, 7250, 2543, 9091, 8820, 9129, 2616, 8202, 6405, 8678, 8056, 7845, 1469, 148, 3574, 7622, 6473, 8129, 4354, 6287, 1546, 341, 4226, 8006, 4056, 2297, 6798, 3853, 394, 7978, 2231, 6426, 9126, 8694, 2119, 1996, 203, 1340, 2528, 3762, 8466, 3645, 7393, 6928, 7451, 4960, 1388, 3728, 2350, 10165, 7742, 6162, 9005, 7252, 8987, 2372, 6790, 437, 6868, 6505, 2345, 10886, 4223, 6831, 6472, 6302, 10146, 6416, 2105, 8078, 6170, 2269, 10012, 6645, 1636, 3499, 4262, 7444, 10974, 10936, 1164, 8589, 2537, 7330, 4231, 9140, 2634, 1403, 8502, 3913, 7661, 4174, 5127, 7678, 6055, 7777, 4799, 3706, 1903, 3866, 9145, 9042, 7853, 8985, 6855, 4209, 11791, 1421, 2661, 4249, 7959, 1661, 8251, 1993, 7662, 6493, 8155, 2222, 6502, 6811, 382, 8868, 4040, 7344, 8677, 7238, 10091, 6043, 8846, 6775, 2656, 6190, 8244, 6829, 7664, 7124, 8122, 9479, 9079, 2730, 3601, 1273, 8247, 4849, 5345, 2097, 7636, 5073, 3633, 6833, 6069, 7729, 3873, 8975, 7809, 11056, 1484, 4998, 2359, 1632, 8496, 11009, 9019, 8769, 8815, 4730, 3945, 133, 6553, 11025, 8436, 6431, 6919, 1526, 9115, 1329, 9009, 9087, 7572, 6022, 10029, 10415, 8063, 3802, 11103, 7181, 7148, 1424, 3942, 1574, 2762, 7334, 4026, 7595]


myanmar_test_ix=[6709, 7867, 8889, 6425, 1955, 1752, 7790, 7550, 7806, 6247, 2389, 8594, 2463, 6873, 10525, 1443, 6556, 6824, 1770, 10100, 8795, 6789, 1578, 6818, 3899, 8806, 3620, 7446, 2795, 10911, 4031, 2109, 7152, 10999, 8929, 6216, 2640, 8561, 7097, 10122, 10905, 7924, 6765, 8198, 4269, 5682, 1634, 8888, 9076, 8655, 8566, 8574, 6262, 7743, 7335, 8127, 6024, 7832, 6640, 6565, 6792, 8870, 1505, 9120, 7388, 7804, 7620, 2007, 1345, 8845, 10524, 7244, 1477, 11003, 4878, 8, 6418, 10518, 1206, 7888, 7381, 99, 8008, 6554, 2012, 6443, 11871, 9134, 9355, 11731, 3771, 6751, 8994, 8059, 313, 5110, 9168, 6172, 10173, 3948, 6460, 10140, 3679, 10487, 7416, 2193, 7570, 1267, 8970, 9056, 1214, 2425, 4196, 2260, 7592, 1466, 8052, 3797, 8030, 1565, 8236, 7218, 2810, 7338, 6871, 9982, 7606, 3944, 7475, 10078, 6108, 1293, 4867, 2337, 8887, 5346, 8029, 9106, 2830, 2243, 6513, 8739, 6515, 8415, 7969, 8903, 10928, 8258, 7713, 6814, 8439, 1652, 7494, 8725, 6500, 7760, 10526, 10902, 8135, 7955, 6045, 3675, 9973, 7102, 2340, 6187, 10111, 8885, 3677, 9290, 10057, 2293, 11890, 7219, 6774, 2509, 10514, 7092, 9171, 106, 3975, 6468, 8601, 4793, 2626, 14, 8162, 9158, 4909, 8292, 1215, 2227, 7707, 6209, 3625, 1995, 3891, 1535, 9241, 4796, 6278, 83, 98, 3719, 2110, 6531, 7789, 2540, 6773, 4055, 10462, 3712, 357, 3777, 2428, 1595, 2130, 3851, 2116, 10900, 10089, 7632, 8944, 2441, 11805, 8382, 10494, 11112, 8578, 8719, 2083, 7800, 3575, 10950, 7345, 6927, 4244, 7104, 7113, 2614, 7415, 7075, 1440, 1976, 10533, 4050, 7705, 5335, 2322, 9077, 2335, 1973, 3727, 8776, 9108, 8106, 7711, 7324, 7348, 152, 8411, 6887, 10170, 8355, 8848, 8617, 6270, 7349, 1390, 9975, 8951, 10896, 3586, 6552, 6527, 10536, 8748, 1475, 7681, 1173, 7158, 7616, 2021, 7894, 10895, 6686, 7718, 6387, 3699, 8656, 4250, 8064, 6146, 6104, 7433, 6152, 9065, 155, 8793, 3852, 7483, 7491, 45, 7251, 11037, 1243, 6618, 11825, 4859, 7350, 9480, 9104, 8262, 2357, 157, 7202, 10924, 6398, 7522, 8826, 9153, 6923, 8553, 8666, 3453, 8062, 10404, 6734, 4237, 6770, 3804, 10874, 6491, 1563, 2758, 7272, 375, 384, 2710, 6186, 6867, 1253, 3839, 11011, 8789, 7915, 7460, 3579, 6369, 8080, 7642, 9098, 7220, 3915, 1201, 7744, 1618, 8979, 2812, 2458, 4080, 8358, 6546, 3628, 9103, 4208, 3935, 7649, 9993, 7304, 3687, 10515, 8130, 2676, 2155, 8324, 6266, 8628, 10881, 8108, 8443, 7792, 6768, 412, 7843, 1970, 11854, 8592, 8174, 7462, 2621, 8922, 126, 6787, 8945, 7201, 1358, 1558, 3711, 6632, 7166, 7171, 7574, 8369, 2709, 10529, 8115, 11788, 10090, 7361, 7893, 3785, 1549, 2657, 3994, 6903, 1404, 1464, 2375, 8435, 8147, 3599, 3813, 8744, 6747, 7775, 9088, 7712, 1132, 2045, 7901, 8121, 3567, 8816, 171, 6335, 7227, 2265, 6549, 2294, 1431, 9090, 57, 5426, 2679, 8701, 8464, 7638, 291, 7067, 8172, 1288, 8703, 10496, 7230, 7293, 4797, 8680, 8734, 1587, 6292, 11771, 4092, 7516, 1425, 3535, 2223, 8276, 2232, 10074, 1476, 1570, 4005, 6685, 4684, 24, 2439, 7372, 4199, 6089, 7697, 2407, 5112, 4779, 3900, 4873, 2188, 6548, 1378, 1518, 3870, 2651, 1585, 9475, 5065, 7682, 2228, 10123, 8644, 4830, 8660, 1513, 8895, 6477, 6250, 2245, 1356, 2814, 7456, 2423, 3008, 9203, 11782, 8139, 3643, 11894, 7370, 85, 8942, 3603, 5081, 3967, 8650, 4358, 8372, 3640, 4906, 6040, 3668, 10971, 1187, 8575, 8832, 1371, 6893, 6333, 7876, 10858, 8910, 3985, 10553, 175, 3086, 442, 7144, 6384, 7698, 7543, 6452, 8668, 4836, 11794, 131, 1284, 6914, 7240, 7063, 8455, 11736, 2756, 4362, 8007, 6105, 9081, 7253, 7676, 71, 2739, 282, 49, 2612, 9018, 1237, 1194, 8181, 8225, 6705, 18, 7912, 9068, 8676, 3083, 9173, 2140, 2131, 7142, 8132, 7750, 2288, 11824, 10998, 6118, 6842, 4448, 9064, 2404, 7295, 7306, 2800, 2418, 8771, 8662, 6315, 8898, 1219, 6049, 1831, 8224, 1458, 6666, 1486, 2210, 10097, 9350, 8453, 8366, 8031, 2168, 7688, 7710, 8238, 8560, 7528, 3752, 8189, 5282, 11101, 7050, 7596, 8461, 10017, 8532, 9291, 2358, 8717, 1409, 7146, 2379, 8960, 7999, 7215, 8207, 7566, 6551, 3651, 273, 8934, 3631, 7835, 2452, 8120, 1427, 7667, 2344, 6924, 8019, 3078, 7131, 8886, 7101, 387, 8658, 6729, 42, 6532, 3733, 249, 1224, 8625, 7119, 7071, 4703, 8569, 4203, 8214, 8305, 11593, 8590, 4061, 6442, 2781, 2741, 6577, 8234, 1750, 1596, 7091, 4015, 11139, 2719, 4218, 8142, 59, 10942, 2094, 10043, 1342, 6675, 6440, 7759, 10179, 7539, 6603, 9037, 7143, 8735, 6715, 7496, 8802, 6390, 7508, 2067, 6410, 1956, 1609, 8196, 6168, 7966, 10005, 2806, 7567, 6512, 7552, 3946, 8058, 6297, 8737, 8088, 4248, 3872, 2690, 7635, 1692, 3828, 10961, 1112, 2041, 7204, 2732, 1830, 11128, 7945, 3686, 10025, 7474, 1191, 6576, 6393, 8200, 8334, 4296, 2696, 7428, 8352, 1393, 3585, 3876, 1965, 1372, 6485, 1972, 7331, 5095, 7584, 1562, 7905, 7309, 2200, 3767, 8890, 9159, 7089, 4687, 7369, 7300, 113, 3906, 8310, 9160, 4334, 9054, 1420, 6161, 9053, 3792, 4682, 6053, 10484, 3683, 2302, 6376, 8256, 7184, 10850, 418, 7590, 1640, 6229, 8708, 7090, 1400, 181, 1498, 9996, 8220, 6909, 2827, 6711, 9127, 10949, 7916, 10975, 2237, 11829, 8023, 4744, 3930, 10482, 257, 10925, 11760, 10059, 9152, 8216, 7643, 8579, 1630, 1553, 7831, 9092, 8163, 7870, 3618, 7685, 1294, 4029, 2239, 9136, 7647, 9142]

print(len(myanmar_train_ix))


def convert_train_test_split(split, removed_column):
    removed_column.sort()
    for i in range(len(split)):
        flag = False
        for j in range(len(removed_column)):
            if split[i] < removed_column[j]-1:
                split[i] -= j
                flag = True
                break
            elif split[i] > removed_column[j]-1:
                pass
            else:
                raise 'wrong split'
        if not flag:
            split[i] -= 10
    return split


3360


In [ ]:
class SingleLayerMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout):
        super(SingleLayerMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.sigmoid(x)
        return x

class TwoLayerMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout):
        super(TwoLayerMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.sigmoid(self.fc2(x))
        return x

class FourLayerMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout):
        super(FourLayerMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, output_size)
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.sigmoid(self.fc4(x))
        return x

        # Custom Dataset
class MyDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

class CompositeLoss(nn.Module):
    def __init__(self, l1_weight):
        super(CompositeLoss, self).__init__()
        self.mse_loss = nn.MSELoss()
        self.l1_loss = nn.L1Loss()
        self.l1_weight = l1_weight

    def forward(self, y_true, y_pred, model):
        mse = self.mse_loss(y_true, y_pred)
        l1_reg = torch.tensor(0., requires_grad=True)
        for param in model.parameters():
            l1_reg = torch.norm(param, p=1) + l1_reg
        total_loss = mse + self.l1_weight * l1_reg
        return total_loss

# Hyperparameters

def train(model, criterion, optimizer, dataloader, dataloader_test, epochs, scheduler, trial, optimize=True):
    model.train()
    for epoch in range(epochs):
        loss_record = []
        all_prediction = []
        all_groundtruth = []
        for inputs, targets in dataloader:
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            all_groundtruth += targets.squeeze().tolist()
            all_prediction += outputs.squeeze().tolist()

            # Calculate loss
            if isinstance(criterion, CompositeLoss):
                loss = criterion(targets, outputs, model)
            else:
                loss = criterion(outputs, targets)
            loss_record.append(loss.item())
            #print(loss.item())
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            # print('loss: ', loss.item())
        # Print loss every 100 epochs
        scheduler.step()
        mean_loss = sum(loss_record) / len(loss_record)
        # if (epoch + 1) % 1 == 0:
        # print(f"Epoch [{epoch + 1}/{epochs}], Loss: {mean_loss:.4f}, lr: {scheduler.get_last_lr()[0]:.4f}")
        if not optimize:
            wandb.log({"Train Loss": mean_loss, "Learning Rate": scheduler.get_last_lr()[0]})

        mse, r2_value, adj_r2 = test(model, dataloader_test, criterion, optimize)

        # Report an objective function value for a given step.
        # The reported values are used by the pruners to determine whether this trial should be pruned.
        # if optimize:
        #     trial.report(mse, epoch)

        # Handle pruning based on the intermediate value.
        # if trial.should_prune():
        #     raise optuna.exceptions.TrialPruned()
    if not optimize:
        torch.save(model, os.path.join('model', args.feature + '.pth'))
        with open(os.path.join('model', args.feature + '_train.csv'), 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['groundtruth', 'prediction'])
            for i in range(len(all_groundtruth)):
                writer.writerow([all_groundtruth[i], all_prediction[i]])

        layers = 0
        if trial['n_layer'] == 'one':
            layers = 1
        elif trial['n_layer'] == 'two':
            layers = 2
        elif trial['n_layer'] == 'four':
            layers = 4
        df1 = pd.DataFrame({'name': 'MLP+Bayes', 'R2': r2_value, 'adj_R2': adj_r2, 'mean val MSE': 0, 'test MSE': mse*16,
                            'MLP layers': layers, 'MLP hidden layers': layers-1, 'MLP hidden size': trial['hidden_size'],
                            'MLP learning rate': trial['learning_rate'],'MLP batch size': trial['batch_size']
                            #'loss type': trial['loss'],
                            #'norm weight': trial['norm_weight']
                            #'MLP dropout': trial['dropout'],
                            }, index=[0])
        if not os.path.exists('results.xlsx'):
            df1.to_excel('results.xlsx', sheet_name=args.feature.replace(',', '+'), index=False)
        else:
            with pd.ExcelWriter('results.xlsx', mode='a', engine='openpyxl') as writer:
                df1.to_excel(writer, sheet_name=args.feature.replace(',', '+'), index=False)
    return mse

def test(model, dataloader, criterion, optimize):
    model.eval()
    ground_truth = []
    predictions = []
    with torch.no_grad():
        loss_record = []
        for inputs, targets in dataloader:
            outputs = model(inputs)
            if isinstance(criterion, CompositeLoss):
                loss = criterion(targets, outputs, model)
            else:
                loss = criterion(outputs, targets)
            loss_record.append(loss.item())
            ground_truth += targets.squeeze().tolist()
            predictions += outputs.squeeze().tolist()
        # mean_loss = sum(loss_record) / len(loss_record)
        # wrong r2
        # slope, intercept, r_value, p_value, std_err = linregress(ground_truth, predictions)
        r2_value = np.var(predictions) / np.var(ground_truth)
        adj_r2 = 1 - (1 - r2_value) * (len(ground_truth) - 1) / (len(ground_truth) - inputs.shape[1] - 1)
        squared_errors = [(ground_truth[i] - predictions[i]) ** 2 for i in range(len(ground_truth))]
        mse = (sum(squared_errors) / len(predictions))*121
        # print(f"Test Loss: {mse:.4f} R^2: {r_value ** 2:.4f}")
        print(loss.item())
        if not optimize:
            wandb.log({"Test Loss": mse, "R^2": r2_value, "Adj R^2": adj_r2})
            with open(os.path.join('model', args.feature + '_test.csv'), 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['ground_truth', 'prediction'])
                for i in range(len(predictions)):
                    writer.writerow([ground_truth[i], predictions[i]])
        return mse, r2_value, adj_r2

In [ ]:
    data = pd.read_excel("/content/drive/MyDrive/Myanmar/sample size checks myanmar_master_wt_econsec_cnfdistance_new.xlsx")
    selected_feature = ['wealth', 'population', 'AidPerTS', 'Distance_to_Hard_to_reach_towns_(m)', 'Distance_to_violent_Conflicts(m)',
                        'fem_benefit', 'participate', 'membership', 'fem_memb', 'gr_submit', 'gr_resolved', 'labour_days', 'wages', 'toseaportdistance(m)']
    selected_feature += ['ln_aid_capita']
    data.shape
    data.head()
    cols = data.columns.tolist()

In [ ]:
## DIFFERENT MODEL PREDICTOR VARIABLES
M1 = ["wealth", "population", "AidPerTS", "Distance_to_Hard_to_reach_towns_(m)",'Distance_to_violent_Conflicts(m)']
M2 = [ "fem_benefit", "participate", "membership", "fem_memb", "gr_submit", "gr_resolved", "labour_days", "wages"] #village capacity
M3 = cols[cols.index("sp_main")+1:cols.index("sp_sub")] #projects they have
M3.append('econsec_new')
M3.append('socialsec_new')
M4 = cols[cols.index("ethnic")+1:cols.index("state")]+cols[cols.index("state")+1:cols.index("light")]   #ethinicity and states
M5 = cols[cols.index("light"):cols.index("township_admin_3")]+cols[-11:-9]
M5.append('toseaportdistance(m)')   #distance geo features


In [ ]:
def objective(trial, optimize=True, cv=5):
    if optimize:
        hidden_size = trial.suggest_int('hidden_size', 16, 256, 16)
        dropout = trial.suggest_float('dropout', 0.2, 0.8)
        batch_size = trial.suggest_int('batch_size', 16, 256, 16)
        learning_rate = trial.suggest_float('learning_rate', 0.001, 0.2)
        layer = trial.suggest_categorical("n_layer", ["one", "two", "four"])

    else:
        wandb.init(project='Myanmar-MLP-optuna', name=args.feature + "_final", config=args)
        hidden_size = trial['hidden_size']
        dropout = trial['dropout']
        batch_size = trial['batch_size']
        learning_rate = trial['learning_rate']
        layer = trial['n_layer']

        wandb.config.hidden_size = hidden_size
        wandb.config.dropout = dropout
        wandb.config.batch_size = batch_size
        wandb.config.learning_rate = learning_rate
        wandb.config.layer = layer

    acc_summary = []
    if optimize:
        train_valid_split = split_train_val_cv(myanmar_train_ix, cv)
    for i in range(cv):
        if optimize:
            train_ix = []
            for j in range(len(train_valid_split)):
                if j != i:
                    train_ix += train_valid_split[j]
            test_ix = train_valid_split[i]
        else:
            train_ix = myanmar_train_ix
            test_ix = myanmar_test_ix


        feature_candidate = []                  #new list conbination of different features because diff models require diff features
        for item in args.feature.split(','):
            if item == 'M1':
                feature_candidate.append(data[M1])
            elif item == 'M2':
                feature_candidate.append(data[M2])
            elif item == 'M3':
                feature_candidate.append(data[M3])
            elif item == "M4":
                feature_candidate.append(data[M4])
            elif item == "M5":
                feature_candidate.append(data[M5])
            else:
                raise ValueError('Wrong feature name')

        features = np.concatenate(feature_candidate, axis=1)

        ln_aid_capita_index = data[['ln_aid_capita']].values

        features_train = torch.tensor(features[train_ix], dtype=torch.float32)
        features_test = torch.tensor(features[test_ix], dtype=torch.float32)
        labels_train = ((torch.squeeze(torch.tensor(ln_aid_capita_index[train_ix], dtype=torch.float32)))-4)/11
        labels_test = ((torch.squeeze(torch.tensor(ln_aid_capita_index[test_ix], dtype=torch.float32)))-4)/11

        feature_mean = features_train.mean(dim=0)
        feature_std = features_train.std(dim=0)
        features_train = (features_train - feature_mean) / feature_std
        features_test = (features_test - feature_mean) / feature_std


        input_size = features_train.size(1)
        if not optimize:
            wandb.config.input_size = input_size
        # hidden_size = int(args.hidden_size)
        output_size = 1
        # learning_rate = args.learning_rate
        epochs = args.epochs

        # batch_size = args.batch_size
        if layer == 'one':
            model = SingleLayerMLP(input_size, hidden_size, output_size, dropout=dropout)
        elif layer == 'two':
            model = TwoLayerMLP(input_size, hidden_size, output_size, dropout=dropout)
        elif layer == 'four':
            model = FourLayerMLP(input_size, hidden_size, output_size, dropout=dropout)
        criterion = nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
        scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

        # Create DataLoader
        dataset = MyDataset(features_train, labels_train)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, worker_init_fn=worker_init_fn)
        dataset_test = MyDataset(features_test, labels_test)
        dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, worker_init_fn=worker_init_fn)

        # Training loop
        acc = train(model, criterion, optimizer, dataloader, dataloader_test, epochs, scheduler, trial, optimize)
        acc_summary.append(acc)
        if not optimize:
            wandb.finish()
        print(acc_summary)
    return sum(acc_summary) / len(acc_summary)

if __name__ == "__main__":
  # params = {
  #     'learning_rate': 0.001,
  #     'batch_size': 64,
  #     'dropout': 0.5,
  #     'hidden_size': 256,
  #     'feature': 'CNN,geo,twitter',
  #     'architecture': 'one',
  # }

  parser = argparse.ArgumentParser(description='Training a model')
  # parser.add_argument('--learning_rate', type=float, default=params['learning_rate'], help='Learning rate for training')
  # parser.add_argument('--batch_size', type=int, default=int(params['batch_size']), help='Batch size for training')
  parser.add_argument('--epochs', type=int, default=200, help='Number of epochs')
  # parser.add_argument('--dropout', type=float, default=params['dropout'], help='Dropout rate')
  parser.add_argument('--feature', type=str,default='M4', help='Random seed')
  # parser.add_argument('--architecture', type=str, default=params['architecture'])
  # parser.add_argument('--hidden_size', type=str, default=int(params['hidden_size']), help='Hidden size')
  parser.add_argument('--seed', type=int, default=1234, help='Random seed')
  parser.add_argument("-f", required=False)


    # Add more hyperparameters as needed
  global args
  args = parser.parse_args()

  torch.cuda.manual_seed_all(args.seed)
  # np.random.seed(args.seed)
  if torch.cuda.is_available():
      torch.cuda.manual_seed(args.seed)
      torch.cuda.manual_seed_all(args.seed)
      torch.backends.cudnn.deterministic = True
      torch.backends.cudnn.benchmark = False

  def worker_init_fn(worker_id):
      np.random.seed(args.seed + worker_id)

  #data = data[selected_feature].dropna()


  #data[selected_feature][data[selected_feature] == np.Inf] = np.NaN  #Ask Andrew why is there an Inf in the data file
  #dft = data[[selected_feature]].dropna()     #if we drop NA, there are only ~4k samples
  #print('dft: ' + dft.shape)

  #data.describe()




In [ ]:
study_name = args.feature
storage_name = f"sqlite:///%s_db.sqlite3"%args.feature
study = optuna.create_study(direction="maximize", study_name=study_name, storage=storage_name, load_if_exists= True)
study.optimize(objective, n_trials=100)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

objective(trial.params, optimize=False, cv=1)

[I 2023-11-09 15:22:45,610] Using an existing study with name 'M4' instead of creating a new one.
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([208])) that is different to the input size (torch.Size([208, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([192])) that is different to the input size (torch.Size([192, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([48])) that is different to the input size (torch.Size([48, 1])). This will likely lead to in

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


[W 2023-11-09 15:22:51,149] Trial 5 failed with parameters: {'hidden_size': 64, 'dropout': 0.4178101802433098, 'batch_size': 208, 'learning_rate': 0.11571379871481187, 'n_layer': 'one'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-10-b93f6abd34ea>", line 94, in objective
    acc = train(model, criterion, optimizer, dataloader, dataloader_test, epochs, scheduler, trial, optimize)
  File "<ipython-input-16-0f141cc3ddca>", line 79, in train
    for inputs, targets in dataloader:
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 630, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 674, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "/

nan
nan
nan
nan


KeyboardInterrupt: ignored